In [1]:
import pandas as pd
import numpy as np

## step 1. Crawling comments from Youtube url

    - input: specific Youtube url 

    - output: 

* reference: [Selenium을 이용해 Youtube 댓글 가져오기](https://somjang.tistory.com/entry/PythonSelenium%EC%9D%84-%EC%82%AC%EC%9A%A9%ED%95%98%EC%97%AC-%EC%9C%A0%ED%8A%9C%EB%B8%8C-%EB%8C%93%EA%B8%80-%EA%B0%80%EC%A0%B8%EC%98%A4%EA%B8%B0

    - 기존 [Youtube-Comment-Crawler](https://github.com/SOMJANG/Youtube_Comment_Crawler)는 Mac에서 돌아가지 않으므로, 이를 보완 

In [2]:
! pip install selenium

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [6]:
! brew install chromedriver

/bin/sh: brew: command not found


In [3]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
import time
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [4]:
# for test new webdriver
chrome_options = webdriver.ChromeOptions()

driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver') # chromedriver 설치 후, /usr/local/bin/으로 옮겨줘야 함 
driver.get('http://www.google.com/xhtml');
time.sleep(5)

search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.submit()
time.sleep(5)

driver.quit()

In [61]:
def crawl_youtube_page_html_sources(url):
    html_sources = []

    for i in range(5):
        driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver') # chromedriver 설치 후, /usr/local/bin/으로 옮겨줘야 함 
        driver.get(url)
        time.sleep(5)
        last_page_height = driver.execute_script("return document.body.scrollHeight")
        # last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            time.sleep(5)
            #driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
            time.sleep(5)
            #new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            new_page_height = driver.execute_script("return document.body.scrollHeight")
            if new_page_height == last_page_height:
                break
            last_page_height = new_page_height


        html_source = driver.page_source
        html_sources.append(html_source)

        driver.quit()
    return html_sources

In [62]:
def get_user_IDs_and_comments(html_sources):
    my_dataframes = []
    for html in html_sources:
        
        soup = BeautifulSoup(html, 'lxml')
        total_comments = soup.find("yt-formatted-string", class_ ="count-text style-scope ytd-comments-header-renderer").text
        print('총', end = '')
        print(total_comments)
        youtube_user_IDs = soup.select('div#header-author > a > span')
        youtube_comments = soup.select('yt-formatted-string#content-text') ##content-text > span:nth-child(1) 
        youtube_likes = soup.select('div#toolbar > span#vote-count-middle')

        str_youtube_userIDs = []
        str_youtube_comments = []
        str_youtube_likes = []

        for i in range(len(youtube_user_IDs)):
            # user id 
            str_tmp = str(youtube_user_IDs[i].text)
            str_tmp = str_tmp.replace('\n', ' ')
            str_tmp = str_tmp.replace('\t', ' ')
            str_tmp = str_tmp.replace('                ',' ')
            str_youtube_userIDs.append(str_tmp)
            # comments
            str_tmp = str(youtube_comments[i].text) 
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('               ', ' ')
            str_youtube_comments.append(str_tmp)
            # likes
            str_tmp = str(youtube_likes[i].text)
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('               ', ' ')
            str_youtube_likes.append(str_tmp)
            
        pd_data = {"ID":str_youtube_userIDs, "Comment":str_youtube_comments, "Like": str_youtube_likes }
        youtube_pd = pd.DataFrame(pd_data)
        my_dataframes.append(youtube_pd)
        
    return my_dataframes

In [68]:
def crawling_url(url):
    html_sorces = crawl_youtube_page_html_sources(urls)
    my_df = get_user_IDs_and_comments(html_sorces)
    df = my_df[0]
    for i in range(1,len(my_df)):
        df = pd.concat([df, my_df[i]])
    df = df.drop_duplicates()        
    return df 

In [64]:
urls = 'https://www.youtube.com/watch?v=zuULD1jz50c'
cmts = crawling_url(urls) # comments output after crawling

총댓글 207개
총댓글 207개
총댓글 207개
총댓글 207개
총댓글 207개


In [69]:
cmts = cmts.drop_duplicates()

In [70]:
cmts

,ID,Comment,Like
0,주녀니,"엊그제 22000명, 500명, 어제 14000명, 450명 / 단순 비율 계산을 ...",47
1,허진혁,경제경제 거릴거면 기준왜만든건데 ㅋㅋ2.5단계 기준넘었으면 입 다물고 격상해라 버티지말고,40
2,개숙자,제발 줄었다는 뉴스 내보내지 말아주세요 이런뉴스보고 사람들 또 활보합니다--,11
3,이민한,주말은 검사소 자체가 반이싱 쉰다,45
4,은꽃,검사 숫자 대비 확진자 발표를 하셔야죠5000명 검사중 450명 확진인지7000명 ...,20
...,...,...,...
100,임형,"일요일,월요일 발표자료는 주말에 검사자가 적어서 확진자도 적다",5
101,"의병장,조작된 우리역사 찾기",제주는 하루 10만명이상이 들어오는데.. 확진자가 왜 없을까요? 그건 바로 동선공개...,5
102,지혜,K방역 으샤으샤 화이팅 !!,1
103,하하늘과 나,윤석열 코로나 균 처럼 징글징글하다 찌질하게 놀지말고 사표 쓰고 국민의힘 당 으로 ...,1
